In [2]:
from ultralytics import YOLO
import os
import json
from PIL import Image, ImageEnhance, ImageFilter, ImageDraw

model_path = 'watermarks.pt'
INPUT_DIR = "input_images"
OUTPUT_DIR = "output_images"

model = YOLO(model_path)

def image_enhancer(image_path, threshold=70):
	image = Image.open(image_path).convert("RGB")
	gray_image = image.convert("L")
	enhancer = ImageEnhance.Contrast(gray_image)
	contrast_image = enhancer.enhance(0.85)
	sharpened_image = contrast_image.filter(ImageFilter.EDGE_ENHANCE_MORE)
	sharpened_image = sharpened_image.point(lambda x: x if x > threshold else 0)
	return sharpened_image.convert("RGB")

def run_inference(image_paths: list):
    try:
        images = [image_enhancer(image_path) for image_path in image_paths]
        results = [model.predict(image, conf=0.004, iou=0.) for image in images]
        
        saved_paths = [p.replace(INPUT_DIR, OUTPUT_DIR) for p in image_paths]
        
        watermark_status = []
        for idx, result in enumerate(results):
            result = result[0]

            image = Image.open(image_paths[idx])
            if len(result.boxes) > 0:
                watermark_status.append(
                    {
                        "image": image_paths[idx],
                        "status": True
                    }
                )
                for box in result.boxes:
                    coordinates = box.xyxy.tolist()
                    draw = ImageDraw.Draw(image)
                    draw.rectangle(coordinates[0], outline="red", width=3)
                image.save(saved_paths[idx])
            else:
                watermark_status.append(
                    {
                        "image": image_paths[idx],
                        "status": False
                    }
                )
                image.save(saved_paths[idx])

        with open("result.json", "w", encoding='utf-8') as f:
            json.dump(watermark_status, f, indent=4)

    except Exception as e:
        print(e)


In [3]:
image_paths = [os.path.join(INPUT_DIR, p) for p in os.listdir(INPUT_DIR)]
inference_result = run_inference(image_paths)


0: 640x576 8 watermarks, 565.9ms
Speed: 8.7ms preprocess, 565.9ms inference, 17.8ms postprocess per image at shape (1, 3, 640, 576)

0: 448x640 6 watermarks, 161.2ms
Speed: 2.7ms preprocess, 161.2ms inference, 1.2ms postprocess per image at shape (1, 3, 448, 640)
